In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations
from scipy.stats import ttest_ind
from scipy.stats import f_oneway
import numpy as np

params = {'figure.constrained_layout.use': False,
          'axes.facecolor': 'gainsboro',
          'axes.grid': True,
          'grid.linestyle': '--',
          'axes.labelsize': 12,
          'axes.titlesize': 14,
          'xtick.labelsize': 12,
          'ytick.labelsize': 12}
plt.rcParams.update(params)
%matplotlib inline

# Содержание<a name="1"></a>

1. [Информация о датасете](#2)
2. [Очистка данных](#3)
3. [Первичный анализ данных в стобцах](#4)
4. [Корреляционный анализ](#5)
5. [Анализ номинативных переменных](#6)
6. [Выводы](#7)

***

# Информация о датасете<a name="2"></a>

* *__Restaurant_id__* — идентификационный номер ресторана / сети ресторанов;
* *__City__* — город, в котором находится ресторан;
* *__Cuisine Style__* — кухня или кухни, к которым можно отнести блюда, предлагаемые в ресторане;
* *__Ranking__* — место, которое занимает данный ресторан среди всех ресторанов своего города;
* *__Rating__* — рейтинг ресторана по данным TripAdvisor (именно это значение должна будет предсказывать модель);
* *__Price Range__* — диапазон цен в ресторане;
* *__Number of Reviews__* — количество отзывов о ресторане;
* *__Reviews__* — данные о двух отзывах, которые отображаются на сайте ресторана;
* *__URL_TA__* — URL страницы ресторана на TripAdvisor;
* *__ID_TA__* — идентификатор ресторана в базе данных TripAdvisor.

In [11]:
pd.set_option('display.max_rows', 50)  # показывать больше строк
pd.set_option('display.max_columns', 50)  # показывать больше колонок
df = pd.read_csv('main_task_new.csv')

In [12]:
display(df.head(5))
df.info()

,Restaurant_id,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA
0,id_5569,Paris,"['European', 'French', 'International']",5570.0,3.5,$$ - $$$,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643
1,id_1535,Stockholm,NaN,1537.0,4.0,NaN,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032
2,id_352,London,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353.0,4.5,$$$$,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781
3,id_3456,Berlin,NaN,3458.0,5.0,NaN,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776
4,id_615,Munich,"['German', 'Central European', 'Vegetarian Fri...",621.0,4.0,$$ - $$$,84.0,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Restaurant_id      40000 non-null  object 
 1   City               40000 non-null  object 
 2   Cuisine Style      30717 non-null  object 
 3   Ranking            40000 non-null  float64
 4   Rating             40000 non-null  float64
 5   Price Range        26114 non-null  object 
 6   Number of Reviews  37457 non-null  float64
 7   Reviews            40000 non-null  object 
 8   URL_TA             40000 non-null  object 
 9   ID_TA              40000 non-null  object 
dtypes: float64(3), object(7)
memory usage: 3.1+ MB


In [17]:
def col_type(df):
    for column in df.columns:
        print (f'Тип содержимого в колонке {column} - {type(df[column].dropna()[0])}')

In [18]:
col_type(df)

Тип содержимого в колонке Restaurant_id - <class 'str'>
Тип содержимого в колонке City - <class 'str'>
Тип содержимого в колонке Cuisine Style - <class 'str'>
Тип содержимого в колонке Ranking - <class 'numpy.float64'>
Тип содержимого в колонке Rating - <class 'numpy.float64'>
Тип содержимого в колонке Price Range - <class 'str'>
Тип содержимого в колонке Number of Reviews - <class 'numpy.float64'>
Тип содержимого в колонке Reviews - <class 'str'>
Тип содержимого в колонке URL_TA - <class 'str'>
Тип содержимого в колонке ID_TA - <class 'str'>


In [19]:
df['Price Range'].unique()

array(['$$ - $$$', nan, '$$$$', '$'], dtype=object)

In [20]:
df['Reviews'][0]

"[['Good food at your doorstep', 'A good hotel restaurant'], ['12/31/2017', '11/20/2017']]"

[К содержанию](#1)

***

# Очистка данных<a name="3"></a>

[К содержанию](#1)

***

# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели

In [ ]:
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)
X = df.drop(['Restaurant_id', 'Rating'], axis = 1)
y = df['Rating']

In [ ]:
# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split

In [ ]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Создаём, обучаем и тестируем модель

In [ ]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [ ]:
# Создаём модель
regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)

In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))